## Importing and testing

In [79]:
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [80]:
es.ping()

True

In [81]:
employee_es_index = 'test_analyzer_employee_data'

## Checking data

In [82]:
es.search(index=employee_es_index, body={"query": {"match_all": {}}},size=10)

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'test_analyzer_employee_data',
    '_type': '_doc',
    '_id': '1',
    '_score': 1.0,
    '_source': {'first_name': 'John',
     'last_name': 'Smith',
     'age': 25,
     'about': 'I love to go rock climbing',
     'interests': ['sports', 'music']}},
   {'_index': 'test_analyzer_employee_data',
    '_type': '_doc',
    '_id': '2',
    '_score': 1.0,
    '_source': {'first_name': 'Araya',
     'last_name': 'Smith',
     'age': 22,
     'about': 'I love to read books',
     'interests': ['books', 'music']}}]}}

## Searching

### Highlights

In [83]:
body = {
            "query": {
                "match": { 'about': {'query': "rocks"}
                         }

            }
        }

In [84]:
sources = ["about"]

In [85]:
body["highlight"] = {
        "pre_tags": ["<em class='highlight'>"],
        "post_tags": ["</em>"],
        'fields': {x: {'number_of_fragments': 0} for x in sources}
    }

In [86]:
es.search(index=employee_es_index, body=body)

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.9186288,
  'hits': [{'_index': 'test_analyzer_employee_data',
    '_type': '_doc',
    '_id': '1',
    '_score': 0.9186288,
    '_source': {'first_name': 'John',
     'last_name': 'Smith',
     'age': 25,
     'about': 'I love to go rock climbing',
     'interests': ['sports', 'music']},
    'highlight': {'about': ["I love to go <em class='highlight'>rock</em> climbing"]}}]}}

## Search on multiple fields

In [88]:
sources = ['first_name','last_name','age','about','interests']

In [89]:
body = {
            "query": {
                "bool": {
                    "should": [{'match': {'about': {'query': 'books'}}},
                               {'match': {'interests': {'query': 'books'}}},
                              ],
                    "minimum_should_match": 1
                }

            }
        }

In [90]:
body["highlight"] = {
        "pre_tags": ["<em class='highlight'>"],
        "post_tags": ["</em>"],
        'fields': {x: {'number_of_fragments': 0} for x in sources}
    }

In [91]:
es.search(index=employee_es_index, body=body)

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.6462245,
  'hits': [{'_index': 'test_analyzer_employee_data',
    '_type': '_doc',
    '_id': '2',
    '_score': 1.6462245,
    '_source': {'first_name': 'Araya',
     'last_name': 'Smith',
     'age': 22,
     'about': 'I love to read books',
     'interests': ['books', 'music']},
    'highlight': {'about': ["I love to read <em class='highlight'>books</em>"],
     'interests': ["<em class='highlight'>books</em>"]}}]}}

## Filters

In [98]:
body = {
    "query": {
      "bool": { 
          "should": [{'match': {'about': {'query': 'books'}}},
                     {'match': {'interests': {'query': 'books'}}},
           ],
          "minimum_should_match": 1,
          "filter": {
               "range":{"age":{"gte":22}}
               }
            }
          }
      }


In [99]:
body["highlight"] = {
        "pre_tags": ["<em class='highlight'>"],
        "post_tags": ["</em>"],
        'fields': {x: {'number_of_fragments': 0} for x in sources}
    }

In [100]:
es.search(index=employee_es_index, body=body)

{'took': 7,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.6462245,
  'hits': [{'_index': 'test_analyzer_employee_data',
    '_type': '_doc',
    '_id': '2',
    '_score': 1.6462245,
    '_source': {'first_name': 'Araya',
     'last_name': 'Smith',
     'age': 22,
     'about': 'I love to read books',
     'interests': ['books', 'music']},
    'highlight': {'about': ["I love to read <em class='highlight'>books</em>"],
     'interests': ["<em class='highlight'>books</em>"]}}]}}